In [183]:
# Импортируем необходимые объекты
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
import re
import pandas as pd
import sqlite3
from sqlite3 import Error
import os 
import datetime

In [184]:
# ------- Этот блок содержит глобальные переменные, необходимые для работы --------- #

#Далее создается элемент класса Firefox WebDriver для нашего url
#Единый указатель ресурса (англ. Uniform Resource Locator, URL 
# — единообразный локатор (определитель местонахождения) ресурса (в нашем случае сайт Картотеки арбитражных дел)
url = 'http://ras.arbitr.ru/'  
path_to_user_driver = 'C:/Users/besso/Desktop/VKR/chromedriver.exe'
path_to_anticaptcha_plugin = 'C:/Users/besso/Desktop/VKR/anticaptcha-plugin_v0.3410.zip' # Имеется в виду Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали
anti_captcha_key = '3d8da2d26ff3ba9552e5524629cface8'



In [185]:
#Параметры, которые должны быть забиты из приложения !

date_from = ''
date_to = ''
doc_text_input = 'казенное учреждение'
download_directory = 'C:\\Users\\besso\\Desktop\\VKR\\'

#собственно сюда нужно забить необходимые параметры

#C:\Users\besso\Desktop\VKR

In [186]:
# Метод отправки API запроса прямо в плагин
# Например для инициализации API ключа сервиса anti-captcha.com, необходимый для работы плагина
# Работает только на действующей HTML страничке,
# в нашем случае на https://antcpt.com/blank.html
# на страницах вроде about:blank запрос не пройдет
def acp_api_send_request(driver, message_type, data={}):
    message = {
        # всегда указывается именно этот получатель API сообщения
        'receiver': 'antiCaptchaPlugin',
        # тип запроса, например setOptions
        'type': message_type,
        # мерджим с дополнительными данными
        **data
    }
    # выполняем JS код на странице
    # а именно отправляем сообщение стандартным методом window.postMessage
    return driver.execute_script("""
    return window.postMessage({});
    """.format(json.dumps(message)))


In [187]:
def init_chrome_options(path_to_anticaptcha_plugin):
    # Инциируем объект опций для Хрома, чтобы иметь возможность подключить расширение
    options = webdriver.ChromeOptions()
    # Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали ранее
    options.add_extension(path_to_anticaptcha_plugin)
    prefs = {"plugins.always_open_pdf_externally": True, 
            'download.default_directory': download_directory} #Без дополнительных прожатий кнопки "скачать" при открытии pdf-скачиваем его
    options.add_experimental_option("prefs",prefs)
    return options

In [188]:
# Ф-ция для инцииализации веб-драйвера хрома опциями
def get_browser_with_options(path_to_user_driver):
    # Запускаем Браузер (Веб Драйвер Хрома) с указанием места скачивания самого файла драйвера
    browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )
    return browser

In [189]:
# Ф-ция для получения стартовой страницы и обхода капчи на ней
def get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ):
    # Переходим на пустую страницу для выполнения API запроса к плагину
    browser = get_browser_with_options(path_to_user_driver)
    
    browser.get(url)
    
    #здесь делаем закрытие объявления (в случае, если оно появилось)
    #если объявление на сайте не появилось, то Selenium просто не найдет объект, отвечающий за него,
    #и при попытке закрыть объявление сгенерируется исключение
    #чтобы из-за этого исключения не падал весь дальнейший код, делаем обработку исключения с возможностью
    #выполнения остатка кода
    
    try:
        button_close = browser.find_element_by_class_name('b-promo_notification-popup-close')
        button_close.click()
    except NoSuchElementException:
        pass
    
    # Устанавливаем API ключ anti-captcha.com
    # замените YOUR-ANTI-CAPTCHA-API-KEY на Ваш шестнадцатиричный ключ, который можно взять тут:
    # https://anti-captcha.com/clients/settings/apisetup
    
    acp_api_send_request(
        browser,
        'setOptions',
        {'options': {'antiCaptchaApiKey': anti_captcha_key}}
    )

    # Три секунды паузы чтобы плагин проверил ключ на стороне anti-captcha.com
    time.sleep(10)
    
    return browser



In [190]:
# Необходимо добавить функцию, которая будет колбасить курсором рандомно во избежание бана

In [191]:
def human_text_enter( element, text_input ):
    time.sleep(random.uniform(2, 5)) 
    # sending_text = 'мошенничество'  text input 
    #document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') 
    for letter in text_input: 
        time.sleep(random.uniform(0.05, 0.16)) 
        element.send_keys(letter)
    

In [192]:
# Ф-ция, которая забивает форму на сайте нужными нам параметрами (пока параметры не все, потом добавим, сейчас в тестовом режиме)
def get_page_by_input_attributes(browser= get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ), doc_text_in = '', date_from_in = '', date_to_in = '' ):
    
    document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') #текст документа
   # document_text.send_keys(doc_text_in)
    
    human_text_enter(document_text, doc_text_in) # Эмулируем ввод текста документа человеком
    
    #dispute_type = browser.find_element_by_xpath('//input[@placeholder="вид спора"]') #вид спора
    #dispute_category = browser.find_element_by_xpath('//input[@placeholder="категория спора"]') #категория спора
    
    #case_participant = browser.find_element_by_xpath('//textarea[@placeholder="название, ИНН или ОГРН"]') #название, ИНН или ОГРН
    #court = browser.find_element_by_xpath('//input[@placeholder="название суда"]') #название суда
    #court.send_keys(court_in)
    #case_number = browser.find_element_by_xpath('//input[@placeholder="например, А50-5568/08"]') #номер дела
    
    date_from = browser.find_element_by_xpath('//label[@class="from"]//input[@placeholder="дд.мм.гггг"]') #нижняя граница даты
    human_text_enter(date_from, date_from_in) # Эмулируем ввод дат человеком
    
    date_to = browser.find_element_by_xpath('//label[@class="to"]//input[@placeholder="дд.мм.гггг"]') #верхняя граница даты
    human_text_enter(date_to, date_to_in)
    
    search_button = browser.find_element_by_xpath('//button[@alt="Найти"]') #кнопка "Найти"
    
    # Самая важная чаcть: ждем не более 120 секунд пока индикатор антикаптчи с классом antigate_solver
    # не получит класс solved, что означает что рекапча решена
    WebDriverWait(browser, 120)
    
    # Отправляем форму
    search_button.click()
    
    time.sleep(10)
    #ищем ссылку для перехода на страницу документа и переходим по ней
    return browser


In [193]:
initial_parsing_page = get_page_by_input_attributes(doc_text_in =  doc_text_input ) # пока это текущий браузер

In [194]:
def get_row_links_from_1_page(fist_page=initial_parsing_page):
    links = initial_parsing_page.find_elements_by_xpath("//a[@href]")
    all_links = list()
    for link in links:
        all_links.append(link.get_attribute("href"))
    return all_links
   

In [195]:
def get_pdf_links(all_links): 
    pdf_links = list()
    for link in all_links:
        if '.pdf' in link:
            pdf_links.append(link)
        else:
            continue
    return pdf_links


In [196]:
def count_found_documents(intial_page = initial_parsing_page):
    total_count = initial_parsing_page.find_elements_by_xpath('//*[@id="contentHeader"]/span[1]')
    split_text = total_count[0].text.split()
    return int(split_text[1])

In [197]:
def current_doc_info_text(intial_page, n  ):
    pattern_searh = '//*[@id="b-cases"]/li['
    pattern_searh = pattern_searh + str(n) + ']'
    try:
        current_page = initial_parsing_page.find_elements_by_xpath(pattern_searh) # будем итерироваться по индексу
        description = current_page[0].text
    except:
        NoSuchElementException
        pass
    return description

#можем выцепить порядковый номер дела, Номер дела, Инстанция, дата размещения документа , название документа

In [198]:
# Получаем данные для заполнения бд 
def get_doc_attributes(description ):
    ordinal_doc_number = int(description[0 : description.find('.') ]) # порядковый номер дела (нужен для показа загруженных материалов)
    start_pos_case_number = description.find('\n')
    end_pos_case_number = description.find('\n', start_pos_case_number + 1 ) # ищет второе вхождение этого символа в строке
    case_number = description[start_pos_case_number + 1 : end_pos_case_number ] # Номер дела
    try:
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[1] # дата заливки документа
    except:
        IndexError # индекс не входит в диапазон элементов.
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[0] # дата заливки документа
    
    court_name = description[end_pos_case_number + 1 : description.find(date_doc)] # Название суда, выносившего решение

    start_doc_pos = description.find(date_doc) + 10
    doc_name = description[ start_doc_pos : ]
    indexs = [i for i, symb in enumerate(doc_name) if symb=='\n']
    document_name = '' # название документа
    for i in range(len(doc_name)):
        if i not in indexs:
            document_name +=  doc_name[i] 
    
    return ordinal_doc_number, case_number, date_doc, court_name, document_name

    

In [199]:
# Скачиваются все файлы с текущей страницы браузера. Доработать так, чтобы итерировалось еще по другим страницам
def download_description(browser, pdf_links ):
    
    insert_values = { 'ordinal_number' : [], 
                      'case_number' : [],
                      'date_doc' : [], 
                      'court_name' : [],
                      'document_name' : [],
                      'pdf_link' : [],
                    }
    
    cnt_documents = count_found_documents() # Общее количество документов
    
    if cnt_documents > 40 * 25:
        cnt_documents = 40 * 25 # Сайт выдает только 40 страниц по 25 документов 
    
    for i in range(len(pdf_links)):
        
        description = current_doc_info_text(browser, i + 1 ) # получили описание нужного дока 
        
        ordinal_doc_number, case_number, date_doc, court_name, document_name = get_doc_attributes(description = description)
        
        
        # добавить в словарь key - value и расширять его по ключам 
        
        
      #  print("Идет загрузка документа {0} из {1}".format(ordinal_doc_number, cnt_documents))
        #print(ordinal_doc_number)
        if pdf_links[i] not in insert_values['pdf_link']:
            insert_values['ordinal_number'].append(ordinal_doc_number)
            #print(case_number)
            insert_values['case_number'].append(case_number)
            #print(date_doc)
            insert_values['date_doc'].append(date_doc)
            #print(court_name)
            insert_values['court_name'].append(court_name)
            #print(document_name)
            insert_values['document_name'].append(document_name)
            #print(pdf_links[i]) # просто линк текущего документа
            insert_values['pdf_link'].append(pdf_links[i])
        #time.sleep(random.uniform(2, 3))  - раскомментить
        else:
            continue
    return insert_values
       
    
    
   

In [200]:
def parse_all_documents(initial_parsing_page):
    cnt_documents = count_found_documents(initial_parsing_page) # Общее количество документов
    # Проход по всем страницам 
    flag = True
    i = 1 # для теста нужно заменить, чтобы было не с 1 страницы 
    x_path = '//*[@id="pages"]/li'
    current_page = initial_parsing_page
    links_need_to_be_downloaded = list()
    list_of_insert_values = list()
    while flag and i <= 12 : # убрать вторую часть условия для того, чтобы был парсинг ВСЕХ  страниц 
            #print(i)
            if i < 10:
                try :
                    print("page nom ", i)
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i + 2) + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print(x_path + '[' + str(i + 1) + ']/a')
                   # print("ya tut 1 ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page = success")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links = success")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    list_of_insert_values.append(insert_values)
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(i + 1) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))         
                except : 
                    NoSuchElementException
                    flag = False  
            
           
            elif i % 10 == 0:
                try:
                    if i == 10: 
                        j = 11
                    else :
                        j = 13 
                        
                        
                        

                
                    #print(x_path + '[' + str(i + 1) + ']/a' + 'elif')
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(j) + ']/a') # После парсинга текущей страницы переходим к следующей
                
                    
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page- succ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - succ ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - succ ")
                    list_of_insert_values.append(insert_values) 
                    #print("append - succ ")
                    
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(j) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))          
                    next_page = initial_parsing_page.find_element_by_xpath(x_path + '[' + str(j + 1 ) + ']/a') # После парсинга текущей страницы переходим к следующей
                    next_page.click()
                    time.sleep(random.uniform(5, 10))  

                except:
                    NoSuchElementException
                    flag = False
                    
                
                
                
            else:
                try:
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i % 10  + 3)  + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print("np=", next_page)
                    #print("ya tut ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page - else block  ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - else block  ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - else block  ")
                    
                    # добавить проверку на то, что если есть такая ссылка, то не добавлять ее в список!
                    
                    
                    list_of_insert_values.append(insert_values)
                    #print("append - else block  ")
                    #download_docs(browser, pdf_links)
                    
                    next_page.click()
                    #print("click - else block  ")
                    i += 1 
                    time.sleep(random.uniform(5, 10))        
                except : 
                    NoSuchElementException
                    flag = False
    # Если мы здесь - значит следующую страницу найти нельзя ( значит надо обработать текущую, т.е. последннюю )
    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
    links_need_to_be_downloaded += pdf_links       
    insert_values = download_description(initial_parsing_page, pdf_links)
    list_of_insert_values.append(insert_values)
    initial_parsing_page.quit() 
                    
                    
          
        
    return links_need_to_be_downloaded, list_of_insert_values
                
                
                

In [201]:
all_needed_links, list_of_insert_values = parse_all_documents(initial_parsing_page=initial_parsing_page)
# Можно подавить вывод информации, которая появляется ниже, просто закомментив все print -ы, но для отладки так нагляднее

page nom  1
page nom  2
page nom  3
page nom  4
page nom  5
page nom  6
page nom  7
page nom  8
page nom  9


In [202]:
res_insert_values = pd.concat([pd.DataFrame(d) for d in list_of_insert_values]).to_dict('list')


In [203]:
for i in res_insert_values['ordinal_number']:
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
251
252


In [204]:
print(len(res_insert_values['ordinal_number']))
print(len(res_insert_values['case_number']))
print(len(res_insert_values['court_name']))
print(len(res_insert_values['document_name']))

check_list = list()
for i in res_insert_values['ordinal_number']:
    if i not in check_list:
        check_list.append(i)
    else:
        continue
len(check_list)

325
325
325
325


300

In [179]:
res_insert_values.keys()

dict_keys(['ordinal_number', 'case_number', 'date_doc', 'court_name', 'document_name', 'pdf_link'])

In [67]:
# сделать проверку на уникальные ключи, чтобы дубли убрались

In [229]:
def del_duplicates(res_insert_values):
    
    unique = list()
    position = 0
    
    for i in res_insert_values['ordinal_number']:
        if i not in unique and (len(unique) < (len(set(res_insert_values['ordinal_number'])))):
            unique.append(i) #отбираем уникальные id-шники
        else:
            res_insert_values['ordinal_number'].pop(position)
            res_insert_values['case_number'].pop(position)
            res_insert_values['date_doc'].pop(position)
            res_insert_values['court_name'].pop(position)
            res_insert_values['document_name'].pop(position)
            res_insert_values['pdf_link'].pop(position)
        position += 1   
            
    df = pd.DataFrame.from_dict(res_insert_values)
    return df

In [230]:
df = del_duplicates(res_insert_values)


In [231]:
df

,ordinal_number,case_number,date_doc,court_name,document_name,pdf_link
0,1,А14-18572/2019,24.12.2020,АС Воронежской области,РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ...,https://ras.arbitr.ru/Kad/PdfDocument/56716b9d...
1,2,А19-4481/2019,21.04.2020,АС Иркутской области,ОБ ОТЛОЖЕНИИ РАССМОТРЕНИЯ ЗАЯВЛЕНИЯ/ЖАЛОБЫ,https://ras.arbitr.ru/Kad/PdfDocument/c8e42cde...
2,3,А60-5329/2020,21.04.2020,АС Свердловской области,[ПОДПИСАНО]РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ...,https://ras.arbitr.ru/Kad/PdfDocument/93a07648...
3,4,А72-2592/2020,21.04.2020,АС Ульяновской области,"РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМ...",https://ras.arbitr.ru/Kad/PdfDocument/4e77f77c...
4,5,А42-11728/2019,21.04.2020,АС Мурманской области,О ВКЛЮЧЕНИИ ТРЕБОВАНИЙ В РЕЕСТР ТРЕБОВАНИЙ КРЕ...,https://ras.arbitr.ru/Kad/PdfDocument/ad60efea...
...,...,...,...,...,...,...
295,296,А60-66730/2019,17.04.2020,17 арбитражный апелляционный суд,ПОСТАНОВЛЕНИЕ СУДА АПЕЛЛЯЦИОННОЙ ИНСТАНЦИИОСТА...,https://ras.arbitr.ru/Kad/PdfDocument/f426d7e5...
296,297,А72-682/2020,17.04.2020,АС Ульяновской области,"МОТИВИРОВАННОЕ РЕШЕНИЕ ПО ДЕЛУ, РАССМОТРЕННОМУ...",https://ras.arbitr.ru/Kad/PdfDocument/c2cbe5e3...
297,298,А03-3829/2020,17.04.2020,АС Алтайского края,О ВОЗВРАЩЕНИИ ИСКОВОГО ЗАЯВЛЕНИЯ (ЗАЯВЛЕНИЯ),https://ras.arbitr.ru/Kad/PdfDocument/ad7dbf55...
298,299,А76-13141/2020,17.04.2020,АС Челябинской области,СУДЕБНЫЙ ПРИКАЗИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ,https://ras.arbitr.ru/Kad/PdfDocument/06d52d84...


In [217]:
len(set(df['ordinal_number']))

300

In [228]:
for i in df['ordinal_number']:
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
# Создание схемы бд  

In [ ]:
name_db = 'my_vkr.db'

In [ ]:
def sql_connection(db_name = name_db):
    try:
        con = sqlite3.connect(name_db) # устанавливаем соединение с БД
        return con
    except Error:
        print(Error)
        

In [ ]:
con = sql_connection()

In [ ]:
def create_tables(connection = con):
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE SEARCH_PARAMS(
                           ID_DOC INTEGER,  
                           SEARCH_DIRECTION TEXT,
                           DATE_FROM TEXT, 
                           DATE_TO TEXT,
                           DOWNLOAD_DATE TEXT,
                           LOCATED_IN_DIRECTORY TEXT ) """)
    con.commit()
    
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE DOC_INFO(
                           ID_DOC INTEGER, 
                           CASE_NUMBER TEXT, 
                           DATE_ON_SITE TEXT, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                           COURT_NAME TEXT, 
                           DOC_TITLE TEXT, 
                           PDF_LINK TEXT) """)
    con.commit()
    
    
    # ЗДЕСЬ ЕЩЕ ДОЛЖНЫ ПОЯВИТЬСЯ 2 СУЩНОСТИ - ТАБЛИЦА С РЕЙТИНГАМИ ДОКОВ ( ПОСЛЕ ТОГО, КАК ОПРЕДЕЛИМСЯ, КАК ОЦЕНИВАЕМ)
    # И ТАБЛИЦА С ЮЗЕРАМИ, КОТОРЫЕ БУДУТ ИМЕТЬ ДОСТУП К БД ИЛИ ТИП ТОГО
    # надо обсудить этот момент
    
    con.close()
    

In [ ]:
try :
    create_tables(connection = con) # создание табличек выдает ошибку если таблички уже существуют 
except:
    print("Table already exists")

In [ ]:
def add_constraints_on_tables(connection  = con):  # пока прототип будущей функции. Нужно согласовать, что будет PK 
    cursor_obj = con.cursor()
    cursor_obj.execute(""" ALTER TABLE SEARCH_PARAMS ADD CONSTRAINT ..... """) 
    
    con.commit()
    
    cursor_obj.execute(""" ALTER TABLE DOC_INFO ADD CONSTRAINT .... """)
    
    con.commit()
    
    con.close()
    
    # первичные ключи определить и еще мб какую-н сущность

In [ ]:
# нужен автоинкремент, чтобы при добавлении в таблицу взять след значение как первичный ключ

In [ ]:
def download_docs( browser, df,  connection = con):
    cursor_obj = con.cursor()
    #browser = get_initial_page()
    n = len(df['ordinal_number'])
    for i in range(len(df['ordinal_number'])):
        print("Загрузка документа {0} из {1}".format(i + 1, n ))
        browser.get(df['pdf_link'][i]) # скачиваем документ по ссылке 
        
        cursor_obj.execute("""
        INSERT INTO DOC_INFO(
                    ID_DOC, 
                    CASE_NUMBER, 
                    DATE_ON_SITE, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                    COURT_NAME, 
                    DOC_TITLE, 
                    PDF_LINK)  
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( df['ordinal_number'][i],
                  df['case_number'][i],
                  df['date_doc'][i],
                  df['court_name'][i],
                  df['document_name'][i],
                  df['pdf_link'][i]) )
        
        time.sleep(1)
        
        now = datetime.datetime.now()
        
        cursor_obj.execute("""
        INSERT INTO SEARCH_PARAMS(
                    ID_DOC,
                    SEARCH_DIRECTION, 
                    DATE_FROM, 
                    DATE_TO,  
                    DOWNLOAD_DATE, 
                    LOCATED_IN_DIRECTORY)
                    
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( df['ordinal_number'][i],
                  doc_text_input,
                  date_from,
                  date_to,
                  now.strftime("%d-%m-%Y %H:%M"),
                  download_directory) )
        
        
        time.sleep(random.uniform(5, 10))
        
        con.commit()
        
    browser.quit()
        
        # Заполнять еще таблицы, о которых упомяналось выше ( с рейтингами доков и юзерами ) 
    

In [ ]:

browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )

download_docs( browser =  browser , df = df ,  connection = con  ) # загрузка доков в указанную папку
